In [42]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as mtl
import autogluon as ag
from autogluon.timeseries import TimeSeriesPredictor,TimeSeriesDataFrame
from autogluon.features.generators import AutoMLPipelineFeatureGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error

In [43]:
df_all = pd.read_csv("Downloads/archive (3)/continuous dataset_timeseries -yearly.csv")
print("There are %0.0f" %df_all.shape[0] + " repeated measures and %0.0f" %df_all.shape[1] +" variables in the dataset" )
df_all.head(4)

There are 48048 repeated measures and 18 variables in the dataset


,datetime,nat_demand,T2M_toc,QV2M_toc,TQL_toc,W2M_toc,T2M_san,QV2M_san,TQL_san,W2M_san,T2M_dav,QV2M_dav,TQL_dav,W2M_dav,Holiday_ID,holiday,school,item_id
0,1/3/2015 0:00,960.3360,25.884526,0.018576,0.016174,21.850546,23.482446,0.017272,0.001855,10.328949,22.662134,0.016562,0.096100,5.364148,0,0,0,2015
1,1/3/2015 1:00,970.3450,25.865259,0.018576,0.016174,21.850546,23.482446,0.017272,0.001855,10.328949,22.662134,0.016562,0.096100,5.364148,0,0,0,2015
2,1/3/2015 2:00,912.1755,25.899255,0.018653,0.016418,22.166944,23.399255,0.017265,0.001327,10.681517,22.578943,0.016509,0.087646,5.572471,0,0,0,2015
3,1/3/2015 3:00,900.2688,25.937280,0.018768,0.015480,22.454911,23.343530,0.017211,0.001428,10.874924,22.531030,0.016479,0.078735,5.871184,0,0,0,2015


In [44]:
df_all['datetime'] = pd.to_datetime(df_all['datetime'],format="%m/%d/%Y %H:%M")

In [45]:
df_all.head(2)

,datetime,nat_demand,T2M_toc,QV2M_toc,TQL_toc,W2M_toc,T2M_san,QV2M_san,TQL_san,W2M_san,T2M_dav,QV2M_dav,TQL_dav,W2M_dav,Holiday_ID,holiday,school,item_id
0,2015-01-03 00:00:00,960.336,25.884526,0.018576,0.016174,21.850546,23.482446,0.017272,0.001855,10.328949,22.662134,0.016562,0.0961,5.364148,0,0,0,2015
1,2015-01-03 01:00:00,970.345,25.865259,0.018576,0.016174,21.850546,23.482446,0.017272,0.001855,10.328949,22.662134,0.016562,0.0961,5.364148,0,0,0,2015


In [46]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48048 entries, 0 to 48047
Data columns (total 18 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   datetime    48048 non-null  datetime64[ns]
 1   nat_demand  48048 non-null  float64       
 2   T2M_toc     48048 non-null  float64       
 3   QV2M_toc    48048 non-null  float64       
 4   TQL_toc     48048 non-null  float64       
 5   W2M_toc     48048 non-null  float64       
 6   T2M_san     48048 non-null  float64       
 7   QV2M_san    48048 non-null  float64       
 8   TQL_san     48048 non-null  float64       
 9   W2M_san     48048 non-null  float64       
 10  T2M_dav     48048 non-null  float64       
 11  QV2M_dav    48048 non-null  float64       
 12  TQL_dav     48048 non-null  float64       
 13  W2M_dav     48048 non-null  float64       
 14  Holiday_ID  48048 non-null  int64         
 15  holiday     48048 non-null  int64         
 16  school      48048 non-

In [47]:
features=AutoMLPipelineFeatureGenerator()
features.fit_transform(df_all)

,nat_demand,T2M_toc,QV2M_toc,TQL_toc,W2M_toc,T2M_san,QV2M_san,TQL_san,W2M_san,T2M_dav,...,TQL_dav,W2M_dav,Holiday_ID,holiday,school,item_id,datetime,datetime.month,datetime.day,datetime.dayofweek
0,960.3360,25.884526,0.018576,0.016174,21.850546,23.482446,0.017272,0.001855,10.328949,22.662134,...,0.096100,5.364148,0,0,0,2015,1420243200000000000,1,3,5
1,970.3450,25.865259,0.018576,0.016174,21.850546,23.482446,0.017272,0.001855,10.328949,22.662134,...,0.096100,5.364148,0,0,0,2015,1420246800000000000,1,3,5
2,912.1755,25.899255,0.018653,0.016418,22.166944,23.399255,0.017265,0.001327,10.681517,22.578943,...,0.087646,5.572471,0,0,0,2015,1420250400000000000,1,3,5
3,900.2688,25.937280,0.018768,0.015480,22.454911,23.343530,0.017211,0.001428,10.874924,22.531030,...,0.078735,5.871184,0,0,0,2015,1420254000000000000,1,3,5
4,889.9538,25.957544,0.018890,0.016273,22.110481,23.238794,0.017128,0.002599,10.518620,22.512231,...,0.068390,5.883621,0,0,0,2015,1420257600000000000,1,3,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48043,1127.2491,27.401392,0.020242,0.068298,8.139692,25.932642,0.019891,0.092651,2.116812,23.885767,...,0.150940,2.431246,0,0,1,2020,1593198000000000000,6,26,4
48044,1128.5592,27.246545,0.020303,0.055511,9.289304,25.715295,0.019746,0.121552,1.990773,23.746545,...,0.150879,2.444658,0,0,1,2020,1593201600000000000,6,26,4
48045,1112.7488,27.099573,0.020395,0.053848,9.837504,25.552698,0.019632,0.153870,2.094459,23.693323,...,0.156311,2.515814,0,0,1,2020,1593205200000000000,6,26,4
48046,1081.5680,26.971155,0.020448,0.057251,10.262464,25.393030,0.019518,0.144531,2.396369,23.658655,...,0.153259,2.800717,0,0,1,2020,1593208800000000000,6,26,4


In [48]:
correlation_matrix = df_all.corr()
correlation_ = correlation_matrix['nat_demand'].sort_values(ascending=False)

In [49]:
print(correlation_)

nat_demand    1.000000
T2M_toc       0.652818
T2M_dav       0.648283
T2M_san       0.627031
W2M_san       0.191757
datetime      0.153810
item_id       0.150870
TQL_san       0.119063
W2M_toc       0.098398
TQL_toc       0.073126
TQL_dav       0.042054
school        0.040094
QV2M_san      0.022193
QV2M_dav     -0.002095
QV2M_toc     -0.036686
W2M_dav      -0.054830
Holiday_ID   -0.129832
holiday      -0.165671
Name: nat_demand, dtype: float64


In [50]:
df_test=df_all[df_all['datetime']>='2020-01-01']

In [51]:
df_test=df_test.drop(['QV2M_dav','QV2M_toc','W2M_dav','TQL_dav','QV2M_san','W2M_toc','TQL_toc','school','Holiday_ID'],axis=1)

In [52]:
df_test.head()

,datetime,nat_demand,T2M_toc,T2M_san,TQL_san,W2M_san,T2M_dav,holiday,item_id
43776,2020-01-01 00:00:00,1032.2630,25.809259,23.301447,0.021866,4.559083,22.598322,1,2020
43777,2020-01-01 01:00:00,1017.5806,25.650842,23.041467,0.023796,4.846412,22.471155,1,2020
43778,2020-01-01 02:00:00,1000.2797,25.501489,22.806177,0.024910,5.204608,22.368677,1,2020
43779,2020-01-01 03:00:00,987.2383,25.430994,22.657556,0.025208,5.628076,22.282556,1,2020
43780,2020-01-01 04:00:00,970.3667,25.360132,22.539819,0.022789,5.729821,22.203882,1,2020


In [53]:
df_train=df_all[df_all['datetime']<'2020-01-01']

In [54]:
df_train.head()

,datetime,nat_demand,T2M_toc,QV2M_toc,TQL_toc,W2M_toc,T2M_san,QV2M_san,TQL_san,W2M_san,T2M_dav,QV2M_dav,TQL_dav,W2M_dav,Holiday_ID,holiday,school,item_id
0,2015-01-03 00:00:00,960.3360,25.884526,0.018576,0.016174,21.850546,23.482446,0.017272,0.001855,10.328949,22.662134,0.016562,0.096100,5.364148,0,0,0,2015
1,2015-01-03 01:00:00,970.3450,25.865259,0.018576,0.016174,21.850546,23.482446,0.017272,0.001855,10.328949,22.662134,0.016562,0.096100,5.364148,0,0,0,2015
2,2015-01-03 02:00:00,912.1755,25.899255,0.018653,0.016418,22.166944,23.399255,0.017265,0.001327,10.681517,22.578943,0.016509,0.087646,5.572471,0,0,0,2015
3,2015-01-03 03:00:00,900.2688,25.937280,0.018768,0.015480,22.454911,23.343530,0.017211,0.001428,10.874924,22.531030,0.016479,0.078735,5.871184,0,0,0,2015
4,2015-01-03 04:00:00,889.9538,25.957544,0.018890,0.016273,22.110481,23.238794,0.017128,0.002599,10.518620,22.512231,0.016487,0.068390,5.883621,0,0,0,2015


In [55]:
columns_to_drop=['QV2M_dav','QV2M_toc','W2M_dav','TQL_dav','QV2M_san','W2M_toc','TQL_toc','school','Holiday_ID']
df_train=df_train.drop(columns_to_drop,axis=1)

In [56]:
df_train.head(10)

,datetime,nat_demand,T2M_toc,T2M_san,TQL_san,W2M_san,T2M_dav,holiday,item_id
0,2015-01-03 00:00:00,960.3360,25.884526,23.482446,0.001855,10.328949,22.662134,0,2015
1,2015-01-03 01:00:00,970.3450,25.865259,23.482446,0.001855,10.328949,22.662134,0,2015
2,2015-01-03 02:00:00,912.1755,25.899255,23.399255,0.001327,10.681517,22.578943,0,2015
3,2015-01-03 03:00:00,900.2688,25.937280,23.343530,0.001428,10.874924,22.531030,0,2015
4,2015-01-03 04:00:00,889.9538,25.957544,23.238794,0.002599,10.518620,22.512231,0,2015
5,2015-01-03 05:00:00,893.6865,25.973840,23.075403,0.001729,9.733589,22.481653,0,2015
6,2015-01-03 06:00:00,879.2323,26.034143,22.995081,0.001485,9.087273,22.456018,0,2015
7,2015-01-03 07:00:00,932.4876,26.691492,24.285242,0.002176,11.395393,22.949304,0,2015
8,2015-01-03 08:00:00,1048.9720,27.674066,26.189691,0.004539,12.872866,24.088129,0,2015
9,2015-01-03 09:00:00,1167.9074,28.760400,27.916650,0.004292,14.548027,25.479150,0,2015


In [57]:
train_data = TimeSeriesDataFrame.from_data_frame(
    df_train,
    timestamp_column="datetime"
)
train_data.head()

nat_demand    T2M_toc    T2M_san   TQL_san  \
item_id timestamp                                                         
2015    2015-01-03 00:00:00    960.3360  25.884526  23.482446  0.001855   
        2015-01-03 01:00:00    970.3450  25.865259  23.482446  0.001855   
        2015-01-03 02:00:00    912.1755  25.899255  23.399255  0.001327   
        2015-01-03 03:00:00    900.2688  25.937280  23.343530  0.001428   
        2015-01-03 04:00:00    889.9538  25.957544  23.238794  0.002599   

                               W2M_san    T2M_dav  holiday  
item_id timestamp                                           
2015    2015-01-03 00:00:00  10.328949  22.662134        0  
        2015-01-03 01:00:00  10.328949  22.662134        0  
        2015-01-03 02:00:00  10.681517  22.578943        0  
        2015-01-03 03:00:00  10.874924  22.531030        0  
        2015-01-03 04:00:00  10.518620  22.512231        0

In [58]:
test_data = TimeSeriesDataFrame(
    df_test,
    timestamp_column="datetime"
)
test_data.head()

nat_demand    T2M_toc    T2M_san   TQL_san  \
item_id timestamp                                                         
2020    2020-01-01 00:00:00   1032.2630  25.809259  23.301447  0.021866   
        2020-01-01 01:00:00   1017.5806  25.650842  23.041467  0.023796   
        2020-01-01 02:00:00   1000.2797  25.501489  22.806177  0.024910   
        2020-01-01 03:00:00    987.2383  25.430994  22.657556  0.025208   
        2020-01-01 04:00:00    970.3667  25.360132  22.539819  0.022789   

                              W2M_san    T2M_dav  holiday  
item_id timestamp                                          
2020    2020-01-01 00:00:00  4.559083  22.598322        1  
        2020-01-01 01:00:00  4.846412  22.471155        1  
        2020-01-01 02:00:00  5.204608  22.368677        1  
        2020-01-01 03:00:00  5.628076  22.282556        1  
        2020-01-01 04:00:00  5.729821  22.203882        1

In [59]:
train_data.shape,test_data.shape

((43776, 7), (4272, 7))

In [60]:
train_data1=train_data.iloc[:-72]
train_data1.tail(24)

nat_demand    T2M_toc    T2M_san   TQL_san  \
item_id timestamp                                                         
2019    2019-12-28 00:00:00   1142.3506  26.767297  25.337610  0.085938   
        2019-12-28 01:00:00   1097.1230  26.649988  25.189050  0.089233   
        2019-12-28 02:00:00   1068.3071  26.547998  25.040186  0.080994   
        2019-12-28 03:00:00   1030.1844  26.443689  24.896814  0.075043   
        2019-12-28 04:00:00   1020.7957  26.358575  24.702325  0.078430   
        2019-12-28 05:00:00   1018.9914  26.311060  24.545435  0.078430   
        2019-12-28 06:00:00   1007.4644  26.298975  24.463037  0.078888   
        2019-12-28 07:00:00   1061.8894  26.857751  25.553064  0.085022   
        2019-12-28 08:00:00   1199.5492  27.639764  26.749139  0.091614   
        2019-12-28 09:00:00   1306.3318  28.319208  27.897333  0.088531   
        2019-12-28 10:00:00   1345.4626  28.903375  28.762750  0.084656   
        2019-12-28 11:00:00   1426.6870  29.326135  29.419885  0.077423   
        2019-12-28 12:00:00   1387.0442  29.566095  29.753595  0.067871   
        2019-12-28 13:00:00   1337.7611  29.712671  29.993921  0.065155   
        2019-12-28 14:00:00   1315.7645  29.689478  30.080103  0.067047   
        2019-12-28 15:00:00   1311.3647  29.462427  30.009302  0.061554   
        2019-12-28 16:00:00   1273.0267  29.097009  29.643884  0.068512   
        2019-12-28 17:00:00   1262.3068  28.476343  28.616968  0.068573   
        2019-12-28 18:00:00   1349.7461  27.810083  27.458521  0.075287   
        2019-12-28 19:00:00   1358.7779  27.479303  26.955865  0.079041   
        2019-12-28 20:00:00   1324.7103  27.260858  26.456171  0.080750   
        2019-12-28 21:00:00   1283.6461  27.115045  26.029108  0.082214   
        2019-12-28 22:00:00   1235.3180  26.993311  25.680811  0.087433   
        2019-12-28 23:00:00   1155.3905  26.895105  25.371667  0.097626   

                              W2M_san    T2M_dav  holiday  
item_id timestamp                                          
2019    2019-12-28 00:00:00  5.090876  23.665735        0  
        2019-12-28 01:00:00  5.264136  23.540613        0  
        2019-12-28 02:00:00  5.178948  23.446436        0  
        2019-12-28 03:00:00  4.927998  23.349939        0  
        2019-12-28 04:00:00  4.309444  23.272638        0  
        2019-12-28 05:00:00  4.073688  23.217310        0  
        2019-12-28 06:00:00  4.156508  23.181787        0  
        2019-12-28 07:00:00  6.026778  23.381189        0  
        2019-12-28 08:00:00  7.575151  23.842889        0  
        2019-12-28 09:00:00  9.398361  24.459833        0  
        2019-12-28 10:00:00  9.345507  25.137750        0  
        2019-12-28 11:00:00  8.237897  25.974573        0  
        2019-12-28 12:00:00  6.945627  26.456720        0  
        2019-12-28 13:00:00  5.811403  26.454858        0  
        2019-12-28 14:00:00  5.266226  26.205103        0  
        2019-12-28 15:00:00  4.628793  25.782739        0  
        2019-12-28 16:00:00  3.376683  25.245447        0  
        2019-12-28 17:00:00  2.404468  24.734155        0  
        2019-12-28 18:00:00  0.895112  24.247583        0  
        2019-12-28 19:00:00  1.013133  23.979303        0  
        2019-12-28 20:00:00  1.474975  23.799921        0  
        2019-12-28 21:00:00  2.124558  23.638483        0  
        2019-12-28 22:00:00  3.080289  23.485498        0  
        2019-12-28 23:00:00  3.989525  23.316980        0

In [61]:
test=train_data.tail(72)
test

nat_demand    T2M_toc    T2M_san   TQL_san  \
item_id timestamp                                                         
2019    2019-12-29 00:00:00   1126.2921  26.819788  25.116663  0.102478   
        2019-12-29 01:00:00   1103.4983  26.740564  24.873376  0.100769   
        2019-12-29 02:00:00   1059.0065  26.623407  24.670282  0.095795   
        2019-12-29 03:00:00   1035.0621  26.521631  24.490381  0.096649   
        2019-12-29 04:00:00   1014.8554  26.477655  24.313593  0.100006   
...                                 ...        ...        ...       ...   
        2019-12-31 19:00:00   1301.6065  26.635645  25.135645  0.064240   
        2019-12-31 20:00:00   1250.9634  26.495935  24.769373  0.058838   
        2019-12-31 21:00:00   1193.6802  26.354456  24.479456  0.038086   
        2019-12-31 22:00:00   1130.4575  26.166895  24.112207  0.020386   
        2019-12-31 23:00:00   1084.4737  25.976373  23.663873  0.019531   

                              W2M_san    T2M_dav  holiday  
item_id timestamp                                          
2019    2019-12-29 00:00:00  4.352335  23.155725        0  
        2019-12-29 01:00:00  4.294046  22.998376        0  
        2019-12-29 02:00:00  4.019113  22.834344        0  
        2019-12-29 03:00:00  3.790254  22.693506        0  
        2019-12-29 04:00:00  3.581013  22.587030        0  
...                               ...        ...      ...  
        2019-12-31 19:00:00  3.086798  23.620020        1  
        2019-12-31 20:00:00  3.659980  23.284998        1  
        2019-12-31 21:00:00  3.769294  23.041956        1  
        2019-12-31 22:00:00  3.872397  22.862207        1  
        2019-12-31 23:00:00  4.165276  22.726373        1  

[72 rows x 7 columns]

In [62]:
test_data.head(10)

nat_demand    T2M_toc    T2M_san   TQL_san  \
item_id timestamp                                                         
2020    2020-01-01 00:00:00   1032.2630  25.809259  23.301447  0.021866   
        2020-01-01 01:00:00   1017.5806  25.650842  23.041467  0.023796   
        2020-01-01 02:00:00   1000.2797  25.501489  22.806177  0.024910   
        2020-01-01 03:00:00    987.2383  25.430994  22.657556  0.025208   
        2020-01-01 04:00:00    970.3667  25.360132  22.539819  0.022789   
        2020-01-01 05:00:00    957.6767  25.316431  22.417993  0.016258   
        2020-01-01 06:00:00    922.3729  25.353662  22.447412  0.013000   
        2020-01-01 07:00:00    924.4692  26.073602  23.831415  0.015656   
        2020-01-01 08:00:00    954.3947  26.938074  25.461511  0.016876   
        2020-01-01 09:00:00    986.0648  27.672021  26.953271  0.014187   

                               W2M_san    T2M_dav  holiday  
item_id timestamp                                           
2020    2020-01-01 00:00:00   4.559083  22.598322        1  
        2020-01-01 01:00:00   4.846412  22.471155        1  
        2020-01-01 02:00:00   5.204608  22.368677        1  
        2020-01-01 03:00:00   5.628076  22.282556        1  
        2020-01-01 04:00:00   5.729821  22.203882        1  
        2020-01-01 05:00:00   5.705475  22.144556        1  
        2020-01-01 06:00:00   5.860416  22.127100        1  
        2020-01-01 07:00:00   8.454858  22.417352        1  
        2020-01-01 08:00:00   9.816476  23.172449        1  
        2020-01-01 09:00:00  11.696469  24.250146        1

In [63]:
#test_data.drop(['nat_demand'],axis=1)

In [64]:
predictor = TimeSeriesPredictor(prediction_length=,target="nat_demand").fit(train_data1,presets="fast_training",hyperparameters="default")

Beginning AutoGluon training...
AutoGluon will save models to 'AutogluonModels\ag-20231220_204353'
=================== System Info ===================
AutoGluon Version:  1.0.0
Python Version:     3.9.18
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22621
CPU Count:          8
GPU Count:          0
Memory Avail:       1.05 GB / 7.86 GB (13.3%)
Disk Space Avail:   90.10 GB / 237.55 GB (37.9%)
Setting presets to: fast_training

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': WQL,
 'hyperparameters': 'default',
 'known_covariates_names': [],
 'num_val_windows': 1,
 'prediction_length': 24,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'target': 'nat_demand',
 'verbosity': 2}

Inferred time series frequency: 'H'
Provided train_data has 43704 rows, 5 time series. Median time series length is 8760 (min=8688, max=8784). 

Provided dataset contains fo

In [ ]:
test_predictions=predictor.predict(test)

In [ ]:
test_predictions.shape

In [ ]:
test_predictions

In [26]:
import autogluon.core
print(autogluon.core.__version__)

1.0.0


In [61]:
predictor.evaluate(test_data)

Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble


{'WQL': -0.04006467079524761}

In [62]:
predictor.leaderboard(test_data)

Additional data provided, testing on additional data. Resulting leaderboard will be sorted according to test score (`score_test`).


,model,score_test,score_val,pred_time_test,pred_time_val,fit_time_marginal,fit_order
0,AutoARIMA,-0.029598,-0.053524,14.334847,66.446332,0.030699,6
1,NPTS,-0.032143,-0.061856,0.580775,1.140532,0.035472,3
2,DynamicOptimizedTheta,-0.033179,-0.057957,0.477460,26.843910,0.023703,5
3,WeightedEnsemble,-0.040065,-0.035728,20.283228,109.708922,1.130691,9
4,DirectTabular,-0.040075,-0.049446,0.204751,0.361147,27.556083,8
5,CrostonSBA,-0.056236,-0.074198,0.031374,15.134327,0.034602,2
6,SeasonalNaive,-0.061432,-0.057140,0.031724,3.934765,0.042898,1
7,AutoETS,-0.067205,-0.072801,3.708568,40.864067,0.020652,4
8,RecursiveTabular,-0.073121,-0.050869,2.035061,2.037376,77.235184,7
